# Добавление списка акваторий в hdf5 файл

Дополнение датасета конфигурационным файлом

In [8]:
import json
import numpy as np
import h5py

In [ ]:
path = "../mt-grabber/positions.json"
dataset_path = "/Volumes/SSD/mark/Documents/Works/MT_Dataset/mt_master_20250714.h5"

In [3]:
positions = []

with open(path, "r", encoding="utf-8") as f:
    positions = json.load(f)

positions

[{'zone': 'Весь регион 0', 'lon': 137.1, 'lat': 40.9, 'zoom': 4},
 {'zone': 'Весь регион 1', 'lon': 145.0, 'lat': 47.7, 'zoom': 5},
 {'zone': 'Весь регион 2', 'lon': 135.7, 'lat': 38.9, 'zoom': 5},
 {'zone': 'Весь регион 3', 'lon': 129.2, 'lat': 34.6, 'zoom': 6},
 {'zone': 'Весь регион 4', 'lon': 136.7, 'lat': 35.6, 'zoom': 6},
 {'zone': 'Весь регион 5', 'lon': 139.7, 'lat': 40.3, 'zoom': 6},
 {'zone': 'Весь регион 6', 'lon': 146.2, 'lat': 44.7, 'zoom': 6},
 {'zone': 'Залив Петра Великого 0', 'lon': 132, 'lat': 42.9, 'zoom': 9},
 {'zone': 'Залив Петра Великого 1', 'lon': 132, 'lat': 43.1, 'zoom': 10},
 {'zone': 'Залив Петра Великого 2', 'lon': 131.901, 'lat': 43.096, 'zoom': 12},
 {'zone': 'Порт Владивосток', 'lon': 131.897, 'lat': 43.101, 'zoom': 14},
 {'zone': 'Бухта Улисс', 'lon': 131.923, 'lat': 43.067, 'zoom': 14},
 {'zone': 'Находка 0', 'lon': 132.7, 'lat': 42.7, 'zoom': 10},
 {'zone': 'Находка 1', 'lon': 132.956, 'lat': 42.781, 'zoom': 12},
 {'zone': 'Сангарский пролив 0', 'lon'

In [ ]:
zone_dtype = np.dtype([
    ("zone_id", "i8"),     # Уникальный идентификатор зоны
    ("name", "S256"),      # Название (описание) зоны
    ("lat", "f4"),         # Широта (WGS-84), в градусах
    ("lon", "f4"),         # Долгота (WGS-84), в градусах
    ("zoom", "i4")         # Уровень зума на карте Marine Traffic
])
zone_id = 0

zones = []

# Перебираем все позиции
for position in positions:
    zones.append((
        zone_id,
        position['zone'].encode('utf-8'),
        float(position['lat']),
        float(position['lon']),
        int(position['zoom'])
    ))

    zone_id += 1

with h5py.File(dataset_path, "a") as f:
    if "zones" not in f:
        ds = f.create_dataset("zones", shape=(0,), maxshape=(None,), dtype=zone_dtype, compression="gzip", chunks=True, compression_opts=4)
    else:
        ds = f["zones"]

    # Увеличиваем размер и дописываем
    old = ds.shape[0]
    ds.resize((old + len(zones),))
    ds[old:] = np.array(zones, dtype=zone_dtype)